In [ ]:
#Her importerer vi modulene som trengs til dette
import json
import requests

#Denne printer bare litt penere...
from pprint import pprint

### Forbered etterspoersel
Her er brukt som eksempel "Gruppering av aarstrinn i grunnskolen 2014-11 - variant av AArstrinn i grunnskolen 2014-11"

In [ ]:
url = 'http://data.ssb.no/api/klass/v1/variants/1121'
headers = {'Accept': 'application/json'}

#Gjoer etterspoersel
r = requests.get(url, headers = headers)

#Av en eller annen grunn returnerer KLASS APIen json som bytestring...
#La oss decode...
cont = r.content.decode('utf8').replace("'", '"')

#Konverter til en liste vi kan bruke
cont = json.loads(cont)

#Print ut resultat
#pprint(cont)

In [ ]:
#Saerbehandling ift. innhold i eksempelrequest
#Om du henter annen data fra Klass saa maa dette sikkert skrives om

#Tittel og info
print(cont['name'])
print('Eies av: %s' % (cont['owningSection']) )
print('Kontaktperson: %s\nEmail: %s\n' % (cont['contactPerson']['name'], cont['contactPerson']['email']))

#Loop gjennom alle klassifikasjoner og print elementer med innhold underveis
for item in cont['classificationItems']:
    code = item['code']
    parentCode = item['parentCode']
    level = item['level']
    name = item['name']
    print( f'Kode: {code}')
    
    #Av og til var denne tom, saa da printer vi den ikke
    if parentCode != "":
        print( f'Foreldrekode: {parentCode}')
    
    print( f'Nivå: {level} \nNavn: {name}\n')

In [ ]:
#Ta ut til json igjen om ønskelig
s = json.dumps(cont, indent = 4 , sort_keys=True)
print(s)

In [ ]:
## Funnet her etter at det over ble skrevet
#https://www.ssb.no/omssb/tjenester-og-verktoy/api/px-api/_attachment/400061?_ts=16dafd88650


# ### Eksempel paa bruk av Klass API fra Python (Klass - Klassifikasjoner og kodelister)
# Se også [API user guide](https://data.ssb.no/api/klass/v1/api-guide.html)

import requests
import pandas as pd
from pandas.io.json import json_normalize #package for flattening json in pandas df


# url til klass API - eksempelet henter Standard for kommuneinndeling med endringer fra 1.1.2017 t.o.m. 1.1.2020
# parameteren includeFuture tar med framtidig inndeling for 2020

URL = 'http://data.ssb.no/api/klass/v1/classifications/131/codes.json?from=2017-01-01&to=2020-01-02&includeFuture=True'
r = requests.get(url = URL)

# r gir bare http status koden
r

# Innholdet finner vi med .text
dataset = r.text

# Leser dataset som json til en dataframe
df = pd.read_json(dataset)

# ### utforsker dataframe
# Dataframe inneholder bare objeketet 'codes'
df.info()
df.columns
# leser codes til pandas Series
ds = df['codes']
# Leser foerste element
ds[0]
# Slutt på utforsking

# Bruker Pandas [json_normalize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html)
kommun = json_normalize(df['codes'])

# om jeg heller ville brukt normalize på ds
# kommun1 = json_normalize(ds)

kommun.head()
kommun.describe()

# setter code som inneholder kommunenummer som index
kommun = kommun.set_index('code')
kommun.tail()

#Finn kommunenummer og gyldighetsperiode for
kommun.query('name == "Holmestrand"')
